In [17]:
import asyncio
# import psycopg
from sqlalchemy.ext.asyncio import create_async_engine, async_sessionmaker, AsyncSession
from sqlalchemy.orm import Session, sessionmaker
from sqlalchemy import text, create_engine, select, MetaData
from config import settings

In [18]:
# Настройка синхронного двигателя
sync_engine = create_engine(
    url=settings.DATABASE_URL_psycopg,
    echo=False,
    pool_size=5,
    max_overflow=10,
)

# Настройка асинхронного двигателя
async_engine = create_async_engine(
    url=settings.DATABASE_URL_asyncpg,
    echo=False,
    pool_size=5,
    max_overflow=10,
)

# Создание фабрик сессий
sync_session = sessionmaker(sync_engine)
async_session = async_sessionmaker(async_engine, expire_on_commit=False)

metadata = MetaData()

In [25]:
async def get_async():
    # Создание асинхронной сессии
    async with async_session() as session:
        async with session.begin():  # Начало транзакции
            # Выполнение запроса
            result = await session.execute(text("SELECT VERSION()"))  
            version = result.scalar_one()  # Получение одного значения
            print(f"Database version: {version}")

            # Повторный запрос для получения строк, если требуется
            result = await session.execute(text("SELECT VERSION()"))  
            first_row = result.first()
            print(f'{first_row=}')  # Первая строка результата (если есть)

            # Итерация по результату
            result = await session.execute(text("SELECT VERSION()"))
            for row in result:
                print(f"Database version: {row[0]}")


In [26]:
# Запуск асинхронной функции
# asyncio.run(get_async())

await get_async()

Database version: PostgreSQL 17.2 (Debian 17.2-1.pgdg120+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 12.2.0-14) 12.2.0, 64-bit
first_row=('PostgreSQL 17.2 (Debian 17.2-1.pgdg120+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 12.2.0-14) 12.2.0, 64-bit',)
Database version: PostgreSQL 17.2 (Debian 17.2-1.pgdg120+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 12.2.0-14) 12.2.0, 64-bit
